# 확률

- 베이지안 통계의 기초는 베이즈 정리이고, 베이즈 정리의 기초는 조건부 확률

<br>

## 01-01. 은행원 린다

- 조건부 확률을 위하여  [Tversky와 Kahneman의 유명한 실험](https://en.wikipedia.org/wiki/Conjunction_fallacy)의 예를 사용:

> Linda는 31세이고 독신이며 솔직하고 밝은 성격을 가지고 있다. 그녀는 철학을 전공했으며, 학창시절에 린다는 차별과 사회적 정의 문제에 심취하였으며, 반핵화 시위에도 참여했다. 이 때, 다음 린다를 설명하는 문장 중 더 가능성 있는 말은 무엇일까?
> 1. Linda는 은행원이다.
> 2. Linda는 은행원이며, 페미니스트 운동에 적극적이다.

많은 사람들이 두 번째 답을 선택하는데, Linda가 **그저** 은행원이기만 한다면 특생이 없어보인다. 그녀가 페미니스트여야 더 말이 되는 것 처럼 보인다.

하지만 두 번째 답은 질문이 요구하는 것처럼 **'더 가능성이 있을'**수 없다. 우리가 그녀의 설명에 부합하는 1,000명의 사람을 찾아서, 그 중 10명이 은행원이라고 한다면, 이 중 페미니스트는 몇 명일까? 많아 봐야 10명 전부 일 것이다. 이런 경우, 두 가지 답은 **동일하게**가능하다. 만약 페미니스트가 10명보다 적다면, 두 번째 답은 가능성이 낮다.

<br>

생물학자 [Stephen J. Gould wrote](https://doi.org/10.1080/09332480.1989.10554932)는 다음과 같이 말했다 :

> 나는 [두 번째] 답이이 가장 개연성이 낮다는 것을 알고 있기 때문에 이 예제를 특히 좋아합니다. 그러나 내 머릿속의 작은[호문쿨루스 (homunculus)](https://en.wikipedia.org/wiki/Homunculus_argument)는 계속해서 뛰어다니면서, '하지만 Linda는 단순한 은행원이 아니야'라고 하고 있기 때문이다.

<br>

<br>

## 01-02. 확률

- 이 시점에서 "확률"의 의미를 정의해야 하는데, 이는 [의외로 어렵다](https://en.wikipedia.org/wiki/Probability_interpretations).  시작하기 전에 막히는 것을 피하기 위해 지금은 간단한 정의를 사용하고 나중에 이를 구체화할 것이다. **확률은 유한 집합의 일부**이다.

- 예를 들어, 1000명을 대상으로 설문조사를 하고 그중 20명이 은행원이라면 은행원으로 일하는 비율은 0.02 또는 2%이다. 이 모집단에서 무작위로 한 사람을 선택하면 은행원일 확률은 2%이다. **"무작위"**란 데이터 세트의 모든 사람이 선택될 확률이 동일함을 의미한다.

In [1]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/data/gss_bayes.csv')

Downloaded gss_bayes.csv


In [2]:
import pandas as pd

gss = pd.read_csv('gss_bayes.csv', index_col=0)
gss.head()

,year,age,sex,polviews,partyid,indus10
caseid,,,,,,
1,1974,21.0,1,4.0,2.0,4970.0
2,1974,41.0,1,5.0,0.0,9160.0
5,1974,58.0,2,6.0,1.0,2670.0
6,1974,30.0,1,5.0,4.0,6870.0
7,1974,48.0,1,5.0,4.0,7860.0


<br>

* `caseid` : 응답자 ID (테이블의 인덱스)

* `year`: 응답자를 조사한 연도

* `age`: 조사 당시 응답자의 나이

* `sex`: 남성 또는 여성

* `polviews`: 진보에서 보수에 이르기까지 다양한 정치적 견해

* `partyid`: 소속 정당 (민주당, 무소속 또는 공화당)

* `indus10`: 응답자가 근무하는 산업에 대한 [코드](https://www.census.gov/cgi-bin/sssd/naics/naicsrch?chart=2007) 

<br>

<br>

## 01-03. 은행원의 비율

- '은행 및 관련분야'코드는 **6870**으로, 다음과 같이 은행원을 찾을 수 있다.

In [3]:
banker = (gss['indus10'] == 6870)
banker.head()

caseid
1    False
2    False
5    False
6     True
7    False
Name: indus10, dtype: bool

In [4]:
banker.sum()

728

- 728명의 은행원이 있음을 확인
- 데이터셋에서 한 사람을 임의로 골랐을 때, 그 사람이 은행원일 확률은 약 1.5%

In [5]:
banker.mean()

0.014769730168391155

<br>

<br>

## 01-04. 확률함수

- boolean에대한 Series객체를 넣으면 확률을 계산하는 함수

In [7]:
def prob(A):
    """Computes the probability of a proposition, A."""    
    return A.mean()

In [8]:
prob(banker)

0.014769730168391155

<br>

- `Sex`열에 대하여, 여성이 나올 확률

```
1    남성
2    여성
```

In [9]:
female = (gss['sex'] == 2)

- 이 데이터셋에서 임의로 한 사람을 선택한 경우, 그 사람이 여성일 확률은 약 54%

In [10]:
prob(female)

0.5378575776019476

<br>

<br>

## 01-05. 정치관과 정당
- 응답자의 정치관을 나타내는 `polviews`

In [15]:
gss['polviews'].unique()

array([4., 5., 6., 2., 7., 3., 1.])

```
1	매우 진보적
2	진보적
3	조금 진보적
4	중립
5	조금 보수적
6	보수적
7	매우 보수적
```

<br>

- `polviews`가 `매우 진보적`, `진보적`, `조금 진보적`중 하나이면, `True`로 표현

In [16]:
liberal = (gss['polviews'] <= 3)

- 이 데이터셋에서 임의로 한 사람을 선택한 경우, 그 사람이 진보성향일 확률은 약 27%

In [17]:
prob(liberal)

0.27374721038750255

<br>

- 응답자의 소속정당을 나타내는 `partyid`

In [18]:
gss['partyid'].unique()

array([2., 0., 1., 4., 5., 7., 3., 6.])

```
0	적극적 민주당원
1	보통 민주당원
2	무소속이나 민주당 지지
3	무소속
4	무소속이나 공화당 지지
5	보통 공화당원
6	적극적 공화당원
7	기타 정당
```

I'll define `democrat` to include respondents who chose "Strong democrat" or "Not strong democrat":

- `partyid`가 `적극적 민주당원`이나 `보통 민주당원`중 하나이면, `True`로 표현

In [20]:
democrat = (gss['partyid'] <= 1)

- 민주당원이라고 응답한 사람들의 비율 약 37%

In [21]:
prob(democrat)

0.3662609048488537

<br>

<br>


## 01-06. 논리곱

- '논리곱'은 `AND`논리 연산자의 다른이름
- **두 개의 명제 A와 B가 있을 때, A와 B모두가 참 $\rightarrow$ 참**


In [27]:
print("응답자가 민주당원일 확률: {:.4f}".format(prob(democrat)))
print("응답자가 은행원일 확률: {:.4f}".format(prob(banker)), end='\n\n')


print("응답자가 민주당원이면서 은행원일 확률: {:.4f}".format(prob(banker & democrat)))
print("응답자가 은행원이면서 민주당원일 확률: {:.4f}".format(prob(democrat & banker)))

응답자가 민주당원일 확률: 0.3663
응답자가 은행원일 확률: 0.0148

응답자가 민주당원이면서 은행원일 확률: 0.0047
응답자가 은행원이면서 민주당원일 확률: 0.0047


<br>

<br>

## 01-07. 조건부 확률

- 조건부 확률은 조건에 따라 달라지는 확률
- 몇 가지 예시

> * 모든 진보 성향 응답자 중, 민주당원일 확률은?
> * 응답자가 은행원인 경우 응답자가 여성일 확률은?
> * 응답자가 여성인 경우 진보적일 확률은?

<br>

- 위 질문에 대해 다음의 두 단계에 걸쳐 확률을 계산 가능


> 1. 모든 진보 성향 응답자를 선택
> 2. 이 중 민주당원의 비율을 계산


In [38]:
selected = democrat[liberal]
print("진보성향 응답자 중 민주당원의 비율: {:.4f}".format(prob(selected)))

진보성향 응답자 중 민주당원의 비율: 0.5206


<br>

- "한 응답자가 은행원일 때, 이 응답자가 여성일 확률"

In [34]:
selected = female[banker]
print("은행원중 여성의 비율: {:.4f}".format(prob(selected)))

은행원 중 여성의 비율: 0.7706


<br>

- 조건부확률을 구하는 함수

In [36]:
def conditional(proposition, given):
    """Probability of A conditioned on given."""
    return prob(proposition[given])

In [40]:
print("응답자가 여성일 때, 진보성향일 확률: {:}".format(conditional(liberal, given=female)))

응답자가 여성일 때, 진보성향일 확률: 0.2758


<br>

<br>

## 01-08. 조건부확률의 비교환성

- `prob(A & B)`는 항상 `prob(B & A)`과 교환가능
- **하지만 조건부확률은 교환이 가능하지 않다. `conditional(A, B)` $\neq$ `conditional(B, A)`**

In [43]:
print("응답자가 여성일때 은행원일 확률: {:.4f}".format(conditional(female, given=banker)))
print("응답자가 은행원일떄 여성일 확률: {:.4f}".format(conditional(banker, given=female)))

응답자가 여성일때 은행원일 확률: 0.7706
응답자가 은행원일떄 여성일 확률: 0.0212


<br>

<br>

## 01-09. 조건과 논리곱


In [45]:
print("응답자가 진보성향의 민주당원일 때, 여성일 확률: {:.4f}".format(conditional(female, given=liberal & democrat)))

응답자가 진보성향의 민주당원일 때, 여성일 확률: 0.5761


In [46]:
print("응답자가 은행원일 때, 진보성향의 여성일 확률: {:.4f}".format(conditional(liberal & female, given=banker)))

응답자가 은행원일 때, 진보성향의 여성일 확률: 0.1731


<br>

<br>

## 01-10. 확률 법칙

<br>

### Theorem 1
* **Theorem 1: 논리곱을 사용한 조건부확률 계산**
  * $P(A)$는 $A$에 대한 확률
<br>

#### $$P(A|B) = \frac{P(A~\mathrm{and}~B)}{P(B)}$$


In [59]:
print("P(A|B) : {:.4f}".format(conditional(female, given=banker)))

print("P(A & B) / P(B) : {:.4f}".format(prob(female & banker) / prob(banker)))

P(A|B) : 0.7706
P(A & B) / P(B) : 0.7706


<br>

### Theorem 2
* **Theorem 2: 조건부확률을 사용한 논리곱 계산**
   * $P(A~\mathrm{and}~B)$는 $A$와 $B$의 논리곱의 확률. **즉 $A$와 $B$가 모두 참일 확률**

<br>

#### $$P(A~\mathrm{and}~B) = P(B) ~ P(A|B)$$

In [57]:
print("P(A & B) : {:.4f}".format(prob(liberal & democrat)))

print("P(A)/P(A|B) : {:.4f}".format(prob(democrat) * conditional(liberal, democrat)))

P(A & B) : 0.1425
P(A)/P(A|B) : 0.1425


<br>

<br>

### Theorem 3
* **Theorem 3: `conditional(A, B)`를 사용한 `conditional(B, A)` 계산**
   * **$P(A | B)$는 $B$가 주어졌을 때 $A$가 참일 확률.**
   * $A$와 $B$ 사이 세로선은 "기븐 (given)"으로 읽음. 
   * **$=$ 베이즈 정리 (Bayes's Theorem)**
   
<br>

- 논리곱은 교환 가능

#### $$P(A~\mathrm{and}~B) = P(B~\mathrm{and}~A)$$

- Theorem 2를 양쪽에 적용시,

<br>

#### $$P(B) P(A|B) = P(A) P(B|A)$$

- 양변을 $P(B)$로 나눌시,

<br>

#### $$P(A|B) = \frac{P(A) P(B|A)}{P(B)}$$

In [61]:
print("P(A | B) : {:.4f}".format(conditional(liberal, given=banker)))

P(A | B) : 0.2239


In [66]:
print("(P(A) P(B|A))/P(B) : {:.4f}".format(prob(liberal) * conditional(banker, liberal) / prob(banker)))

(P(A) P(B|A))/P(B) : 0.2239


<br>

<br>

## 01-11. 전체확률의 법칙

<br>

#### $$P(A) = P(B_1 \mathrm{and} A) + P(B_2 \mathrm{and} A)$$


- **$A$의 전체확률은 두 확률의 합: $B_1$와 $A$의 논리곱이 참이거나, $B_2$의 $A$의 논리곱이 참인 경우**.
- **또한, $B_1$와 $B_2$는 다음을 만족해야 함**:
  * **상호 배제** : 전체 $B$중 하나만 참인 경우

  * **전체 포괄** : 전체 $B$중 하나는 반드시 참

<br>

<br>

#### $$P(A) = P(B_1) P(A|B_1) + P(B_2) P(A|B_2)$$

#### $$P(A) = \sum_i P(B_i) P(A|B_i)$$


In [68]:
print("응답자가 은행원일 확률: {:.4f}".format(prob(banker)))

응답자가 은행원일 확률: 0.0148


In [73]:
male = (gss['sex'] == 1)
print("응답자가 은행원이면서 남성일 확률 + 응답자가 은행원이면서 여성일 확률 = 응답자가 은행원일 확률 = {:.4f}".format(prob(male & banker) + prob(female & banker)))

# (prob(male) * conditional(banker, given=male) + prob(female) * conditional(banker, given=female))

응답자가 은행원이면서 남성일 확률 + 응답자가 은행원이면서 여성일 확률 = 응답자가 은행원일 확률 = 0.0148


## 요약

**Theorem 1** 논리곱을 사용해서 조건부확률 계산:

$$P(A|B) = \frac{P(A~\mathrm{and}~B)}{P(B)}$$

**Theorem 2** 조건부확률을 사용해서 논리곱을 계산:

$$P(A~\mathrm{and}~B) = P(B) P(A|B)$$

**Theorem 3**, 베이즈 정리:

$$P(A|B) = \frac{P(A) P(B|A)}{P(B)}$$

**The Law of Total Probability** 전체확률의 법칙:

$$P(A) = \sum_i P(B_i) P(A|B_i)$$

<br>

## 연습문제

**Exercise1 :** Linda문제의 변형

> Linda는 31세이고 독신이며 솔직하고 매우 밝습니다. 그녀는 철학을 전공했다. 학창 시절 그녀는 차별과 사회 정의 문제에 깊은 관심을 가졌고 반핵 시위에도 참여했습니다. 어느 것이 더 가능성이 있습니까?
> 1. Linda는 은행원입니다.
> 2. Linda는 은행원이며 자신을 민주당원이라고 생각합니다.

<br>

* Linda가 여성 은행원일 확률

In [77]:
prob(female & banker)

0.011381618989653074

* Linda가 진보적인 여성 은행원일 확률

In [78]:
prob(female & banker & liberal)

0.002556299452221546

* Linda가 진보적인 여성 은행원이자 민주당원일 확률

In [79]:
prob(female & banker & liberal & democrat)

0.0012375735443294787

<br>

**Exercise2 :** `conditional`을 사용해서 다음 확률을 계산:

<br>

* 응답자가 민주당원일 때 진보적일 확률은?

In [ ]:
conditional(liberal, given=democrat)

* 응답자가 진보적일 때 민주당원일 확률은?

In [ ]:
conditional(democrat, given=liberal)

<br>

**Exercise3 :** 청년층, 노년층, 진보성향, 보수성향에 대한 [유명한 인용문](https://quoteinvestigator.com/2014/02/24/heart-head/):

> 25살에 자유주의자가 아니면 마음이 없는 것이다. 35세에 보수가 아니면 머리가 없는 것이다.


<br>

- 30세 이하인 경우 `young`으로 정의
- 65세 이상인 경우 `old`로 정의

In [83]:
young = (gss['age'] < 30)
print("prob(young): {:.4f}".format(prob(young)), end='\n\n')

old = (gss['age'] >= 65)
print("prob(old): {:.4f}".format(prob(old)))

prob(young): 0.1944

prob(old): 0.1733


- 이는 대략 20%와 80%에 해당하는 숫자를 고른 조건
- `conservative`라는 변수는 정치적 관점에 '보수적', '조금 보수적', '매우 보수적'이라고 답한 사람들로 정의

In [86]:
conservative = (gss['polviews'] >= 5)
print("prob(conservative): {:.4f}".format(prob(conservative))) 

prob(conservative): 0.3419


<br>

#### `prob`과 `conditional`를 사용하여 다음 확률을 계산

<br>

* 무작위로 선택한 응답자가 젊은 진보주의자일 확률은?


In [87]:
prob(young & liberal)

0.06579427875836884

* 젊은 사람이 진보적일 확률은?

In [88]:
conditional(liberal, given=young)

0.338517745302714

* 응답자가 늙고 보수성향일 확률은?

In [89]:
prob(old & conservative)

0.06701156421180766

* 보수주의자 중 나이가 많은 사람은?

In [90]:
conditional(old, given=conservative)

0.19597721609113564